In [16]:
import cv2
import numpy as np
import os 

In [17]:
folder_path = 'images/roi-motor/SCOOPY'  # Ubah sesuai dengan path folder foto
image_files = os.listdir(folder_path)
count = 0
# print(image_files)

In [18]:
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    frame = cv2.imread(image_path)

    frame = cv2.resize(frame, (frame.shape[1]*4, frame.shape[0]*4))
    cv2.imshow("putih", frame)

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    mask = cv2.inRange(hsv_frame, lower_white, upper_white)

    result = cv2.bitwise_and(frame, frame, mask=mask)
    cv2.imshow("result", result)
    # Temukan kontur pada gambar hasil
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Gambar kotak pada setiap kontur
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        ratio_hFrame_per_hContour = frame.shape[0]/h
        if area > 4000 and area < 18000 and y > 50 and h/w > 0.5:
            print(f"num {count} x:{x} y:{y} w:{w} h:{h} area:{area} ratio:{h/w} hF/hC: {ratio_hFrame_per_hContour}")
            cv2.putText(frame, str(count), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            count += 1

    cv2.imshow('Hasil Deteksi', frame)

    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

num 0 x:50 y:273 w:99 h:116 area:7783.5 ratio:1.1717171717171717 hF/hC: 5.793103448275862
